# 전이학습

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

import torchvision.models as models
import torchvision.datasets as dset
import torchvision.transforms as transforms

from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

In [ ]:
!rm -r images
import os 

try:
  os.mkdir("images")
  os.mkdir("images/dogs")
  os.mkdir("images/cats")
except:
  pass

!wget https://i.kinja-img.com/gawker-media/image/upload/s--WFkXeene--/c_scale,f_auto,fl_progressive,q_80,w_800/ol9ceoqxidudap8owlwn.jpg -P images/dogs
!wget https://www.rspcansw.org.au/wp-content/uploads/2017/08/50_a-feature_dogs-and-puppies_mobile.jpg -P images/dogs
  
!wget https://www.catster.com/wp-content/uploads/2018/05/A-gray-cat-crying-looking-upset.jpg -P images/cats
!wget https://www.scarymommy.com/wp-content/uploads/2018/01/c1.jpg?w=700 -P images/cats

rm: cannot remove 'images': No such file or directory
--2021-04-29 04:32:56--  https://i.kinja-img.com/gawker-media/image/upload/s--WFkXeene--/c_scale,f_auto,fl_progressive,q_80,w_800/ol9ceoqxidudap8owlwn.jpg
Resolving i.kinja-img.com (i.kinja-img.com)... 151.101.194.166, 151.101.130.166, 151.101.66.166, ...
Connecting to i.kinja-img.com (i.kinja-img.com)|151.101.194.166|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 32641 (32K) [image/jpeg]
Saving to: ‘images/dogs/ol9ceoqxidudap8owlwn.jpg’

ol9ceoqxidudap8owlw 100%[===================>]  31.88K  --.-KB/s    in 0.002s  

2021-04-29 04:32:56 (17.9 MB/s) - ‘images/dogs/ol9ceoqxidudap8owlwn.jpg’ saved [32641/32641]

--2021-04-29 04:32:56--  https://www.rspcansw.org.au/wp-content/uploads/2017/08/50_a-feature_dogs-and-puppies_mobile.jpg
Resolving www.rspcansw.org.au (www.rspcansw.org.au)... 101.0.86.38
Connecting to www.rspcansw.org.au (www.rspcansw.org.au)|101.0.86.38|:443... connected.
HTTP request sent, awaitin

In [ ]:
batch_size = 2
learning_rate = 0.001
num_epoch = 10
num_category = 2

In [ ]:
# [Data]
# 1) Load images from folder
img_dir = "./images"
img_data = dset.ImageFolder(img_dir, transforms.Compose([
            transforms.RandomSizedCrop(224),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor()
]))

/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:886: UserWarning: The use of the transforms.RandomSizedCrop transform is deprecated, please use transforms.RandomResizedCrop instead.
  warnings.warn("The use of the transforms.RandomSizedCrop transform is deprecated, " +


In [ ]:
print(img_data)
print(img_data.classes)
print(img_data.class_to_idx)
print(img_data.imgs)

Dataset ImageFolder
    Number of datapoints: 3
    Root location: ./images
    StandardTransform
Transform: Compose(
               RandomSizedCrop(size=(224, 224), scale=(0.08, 1.0), ratio=(0.75, 1.3333), interpolation=bilinear)
               RandomHorizontalFlip(p=0.5)
               ToTensor()
           )
['cats', 'dogs']
{'cats': 0, 'dogs': 1}
[('./images/cats/A-gray-cat-crying-looking-upset.jpg', 0), ('./images/dogs/50_a-feature_dogs-and-puppies_mobile.jpg', 1), ('./images/dogs/ol9ceoqxidudap8owlwn.jpg', 1)]


In [ ]:
# 2) Set data loader
train_loader = DataLoader(img_data,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=2,
                          drop_last=True)

In [ ]:
for img,label in train_loader:
  print(img.size())
  print(label)

torch.Size([2, 3, 224, 224])
tensor([1, 0])


In [ ]:
# [Model]
# 1) ResNet
resnet = models.resnet50(pretrained=True)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth


In [ ]:
for name, moudule in resnet.named_children():
  print(name)

conv1
bn1
relu
maxpool
layer1
layer2
layer3
layer4
avgpool
fc


In [ ]:
# 2) Fully Connected Model

class Resnet(nn.Module):
  def __init__(self):
    super(Resnet, self).__init__()
    self.layer0 = nn.Sequential(*list(resnet.children())[0:-1])
    self.layer1 = nn.Sequential(
        nn.Linear(2048, 500),
        nn.BatchNorm1d(500),
        nn.ReLU(),
        nn.Linear(500, num_category),
        nn.ReLU()
    )
  def forward(self, x):
    out = self.layer0(x)
    out = out.view(batch_size, -1)
    print(out.shape)
    out = self.layer1(out)
    return out

In [ ]:
# 3) Model on GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

model = Resnet().to(device)

for params in model.layer0.parameters():
  params.require_grad = False
for params in model.layer1.parameters():
  params.require_grad = True

cuda:0


In [ ]:
for layer in model.children():
  print(layer)

Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)


In [ ]:
 # 4) Loss & Optimizer
optimizer = optim.Adam(model.layer1.parameters(), lr=learning_rate)
loss_func = nn.CrossEntropyLoss()

In [ ]:
# 5) Train
for i in range(50):
  for j, [image, label] in enumerate(train_loader):
    x = image.to(device)
    y_ = label.to(device)

    optimizer.zero_grad()
    output = model.forward(x)
    loss = loss_func(output, y_)
    optimizer.step()

  if i % 10 == 0:
    print(loss)

torch.Size([2, 2048])
tensor(0.5148, device='cuda:0', grad_fn=<NllLossBackward>)
torch.Size([2, 2048])
torch.Size([2, 2048])
torch.Size([2, 2048])
torch.Size([2, 2048])
torch.Size([2, 2048])
torch.Size([2, 2048])
torch.Size([2, 2048])
torch.Size([2, 2048])
torch.Size([2, 2048])
torch.Size([2, 2048])
tensor(0.8423, device='cuda:0', grad_fn=<NllLossBackward>)
torch.Size([2, 2048])
torch.Size([2, 2048])
torch.Size([2, 2048])
torch.Size([2, 2048])
torch.Size([2, 2048])
torch.Size([2, 2048])
torch.Size([2, 2048])
torch.Size([2, 2048])
torch.Size([2, 2048])
torch.Size([2, 2048])
tensor(0.9300, device='cuda:0', grad_fn=<NllLossBackward>)
torch.Size([2, 2048])
torch.Size([2, 2048])
torch.Size([2, 2048])
torch.Size([2, 2048])
torch.Size([2, 2048])
torch.Size([2, 2048])
torch.Size([2, 2048])
torch.Size([2, 2048])
torch.Size([2, 2048])
torch.Size([2, 2048])
tensor(0.5200, device='cuda:0', grad_fn=<NllLossBackward>)
torch.Size([2, 2048])
torch.Size([2, 2048])
torch.Size([2, 2048])
torch.Size([2, 2

In [ ]:
# 6) Test
model.eval()
correct = 0
total = 0

with torch.no_grad():
  for image, label in train_loader:
    x = image.to(device)
    y_= label.to(device)

    output = model.forward(x)
    _, output_index = torch.max(output, 1)

    total += label.size(0)
    correct += (output_index == y_).sum().float()

  print("Accuracy :", 100*correct/total)

torch.Size([2, 2048])
Accuracy : tensor(100., device='cuda:0')
